# COGS 108 - Final Project 

# Overview

Games nowadays are often categorized fairly broadly, with many falling into the categories of shooters, rpgs, rogue-likes, and so on. Our main goal with this project is to see what type of categories tend to get the best user responses to them and how this response has shifted over time. We wanted to figure out what is the current trend for popular game categories. We will split this up by seeing the trends for each specific year and how the reception for the game has been.

We will utilize two datasets, one for Steam games and one for mobile games in order to see if there's a category more suited for mobile devices compared to computers.

# Names

- Edward Xie

# Group Members IDs

- Edwardxie72---A15534895

# Research Question

What type of game categories get the best user ratings and how has this trend changed over the years. 

Our second goal is to see whether these trends differ between mobile games and PC games.

## Background and Prior Work

## TODO
*Fill in your background and prior work here* 

References (include links):
- 1)
- 2)

# Hypothesis


## TODO
*Fill in your hypotheses here*

# Dataset(s)

*Dataset Information*

- Dataset Name: Steam Store Games
- Link to the dataset: https://www.kaggle.com/nikdavis/steam-store-games
- Number of observations: 27033

This dataset contains 27033 games from steam and it includes information on the games. The data we will be utilizing for each game is the categories of the game, the genre of the game, the steamspy tags of the game, the number of positive ratings for the game, and the number of negative ratings for the game.

- Dataset Name: Google Store Play Apps
- Link to the dataset: https://www.kaggle.com/gauthamp10/google-playstore-apps#Google-Playstore-Full.csv
- Number of observations: 244407

This dataset contains 244407 apps from the play store. With this data set we will extract all the games and from these games, the data we will utilize from them is the category of the game, the rating for the app, and the review count for the app.

*How we will combine the datasets:*
We will be treating these as seperate databases, but will use the results to compare the differences between games on the computer and games on a mobile device to see if there is some difference in the trends between these.


# Setup

In [1]:
#Imports (Following A2 as a guideline for processing data)
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns

In [2]:
# Configure libraries(taken from A2)
# The seaborn library makes plots look nicer
sns.set()
sns.set_context('talk')

# Don't display too many rows/cols of DataFrames
pd.options.display.max_rows = 7
pd.options.display.max_columns = 8

# Round decimals when displaying DataFrames
pd.set_option('precision', 2)

# Data Cleaning

## TODO
Describe your data cleaning steps here.

In [3]:
# Read in data
df_steam = pd.read_csv('steam.csv', low_memory=False)
df_googleplay = pd.read_csv('Google-Playstore-Full.csv', low_memory=False)

In [4]:
# Preview and clean up steam data set first
df_steam.head(5)

,appid,name,release_date,english,...,average_playtime,median_playtime,owners,price
0,10,Counter-Strike,2000-11-01,1,...,17612,317,10000000-20000000,7.19
1,20,Team Fortress Classic,1999-04-01,1,...,277,62,5000000-10000000,3.99
2,30,Day of Defeat,2003-05-01,1,...,187,34,5000000-10000000,3.99
3,40,Deathmatch Classic,2001-06-01,1,...,258,184,5000000-10000000,3.99
4,50,Half-Life: Opposing Force,1999-11-01,1,...,624,415,5000000-10000000,3.99


In [5]:
# Start by dropping unwanted data, keep the name so we have some way to identify apps, but not neccesary
del df_steam['appid']
del df_steam['english']
del df_steam['developer']
del df_steam['publisher']
del df_steam['platforms']
del df_steam['required_age']
del df_steam['achievements']
del df_steam['average_playtime']
del df_steam['median_playtime']
del df_steam['price']
del df_steam['owners']  # We drop owners because it gives a range which might make data inconsistent, instead we will weigh off each rating

In [6]:
# Rename categories before we start cleaning data
df_steam.columns = ['Name', 'Year', 'Steam Categories', 'Genre', 'Steamspy Categories', 'Positive Ratings', 'Negative Ratings']

In [7]:
# Extract only the year from the release date because we analyze by year
def extract_year(string):
    return int(string[:4])  # Only need first 4 characters for year

In [8]:
df_steam['Year'] = df_steam['Year'].apply(extract_year)  # Change all years in our df

In [9]:
# Preview and clean up Google Play Data
df_googleplay.head(5)

,App Name,Category,Rating,Reviews,...,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14
0,DoorDash - Food Delivery,FOOD_AND_DRINK,4.548561573,305034,...,NaN,NaN,NaN,NaN
1,TripAdvisor Hotels Flights Restaurants Attract...,TRAVEL_AND_LOCAL,4.400671482,1207922,...,NaN,NaN,NaN,NaN
2,Peapod,SHOPPING,3.656329393,1967,...,NaN,NaN,NaN,NaN
3,foodpanda - Local Food Delivery,FOOD_AND_DRINK,4.107232571,389154,...,NaN,NaN,NaN,NaN
4,My CookBook Pro (Ad Free),FOOD_AND_DRINK,4.647752285,2291,...,NaN,NaN,NaN,NaN


In [10]:
# Start by dropping unwanted data, keep the name so we have some way to identify apps, but not neccesary
del df_googleplay['Installs']  # We don't care about installs for same reason as steam, use number of ratings instead
del df_googleplay['Size']
del df_googleplay['Price']
del df_googleplay['Content Rating']
del df_googleplay['Last Updated']
del df_googleplay['Minimum Version']
del df_googleplay['Latest Version']
del df_googleplay['Unnamed: 11']
del df_googleplay['Unnamed: 12']
del df_googleplay['Unnamed: 13']
del df_googleplay['Unnamed: 14']

In [11]:
df_googleplay = df_googleplay.dropna(subset=['Category', 'Rating', 'Reviews'])

In [12]:
# First start by dropping all non game category games, games will start with the string "GAME_" in category
df_googleplay = df_googleplay[df_googleplay['Category'].str.contains('GAME')]
df_googleplay = df_googleplay.reset_index(drop=True)

In [13]:
# Add extra columns to steam one for the average rating(out of 10) and total number of ratings(Pos rating/(Pos + neg rating))
df_steam['AvgRating'] = df_steam['Positive Ratings'].div(df_steam['Positive Ratings'].add(df_steam['Negative Ratings']))
df_steam['NumRating'] = df_steam['Positive Ratings'].add(df_steam['Negative Ratings'])

In [14]:
# Add extra columns to googleplay, one for avg rating(out of 10) and rename total to match steam
df_googleplay['Rating'] = df_googleplay['Rating'].astype(float)
df_googleplay['AvgRating'] = df_googleplay['Rating'].mul(2)
df_googleplay.rename(columns = {'Reviews':'NumRating'}, inplace = True) 

In [15]:
# Clean up the categories for the games
df_googleplay['Category'] = df_googleplay['Category'].str.slice(start=5)

In [16]:
# Preview mostly cleaned up data
df_steam.head(5)

,Name,Year,Steam Categories,Genre,...,Positive Ratings,Negative Ratings,AvgRating,NumRating
0,Counter-Strike,2000,Multi-player;Online Multi-Player;Local Multi-P...,Action,...,124534,3339,0.97,127873
1,Team Fortress Classic,1999,Multi-player;Online Multi-Player;Local Multi-P...,Action,...,3318,633,0.84,3951
2,Day of Defeat,2003,Multi-player;Valve Anti-Cheat enabled,Action,...,3416,398,0.90,3814
3,Deathmatch Classic,2001,Multi-player;Online Multi-Player;Local Multi-P...,Action,...,1273,267,0.83,1540
4,Half-Life: Opposing Force,1999,Single-player;Multi-player;Valve Anti-Cheat en...,Action,...,5250,288,0.95,5538


In [17]:
df_googleplay.head(5)

,App Name,Category,Rating,NumRating,AvgRating
0,King of Crabs,ACTION,3.28,785,6.56
1,Match 3 App Rewards: Daily Game Rewards,CASUAL,4.52,248,9.04
2,Brown Dust,ROLE_PLAYING,4.48,70260,8.95
3,Poly - Coloring Puzzle Art Book,PUZZLE,4.58,878,9.16
4,Legend of Empress,ROLE_PLAYING,3.82,750,7.64


In [27]:
# Clean up categories
# Convert Steam Categories + Genre + Steamspy Categories into one category and put it in an array
print(df_steam["Steam Categories"].unique(), len(df_steam["Steam Categories"].unique()))

['Multi-player;Online Multi-Player;Local Multi-Player;Valve Anti-Cheat enabled'
 'Multi-player;Valve Anti-Cheat enabled'
 'Single-player;Multi-player;Valve Anti-Cheat enabled' ...
 'Online Multi-Player;Steam Achievements;Full controller support;In-App Purchases;Steam Cloud'
 'Multi-player;Local Multi-Player;Co-op;Local Co-op;Shared/Split Screen'
 'Multi-player;Online Multi-Player;Cross-Platform Multiplayer;Stats'] 3333


In [26]:
print(df_steam["Genre"].unique(), len(df_steam["Genre"].unique()))

['Action' 'Action;Free to Play' 'Action;Free to Play;Strategy' ...
 'Action;Adventure;Indie;Massively Multiplayer;RPG;Strategy;Early Access'
 'Action;Adventure;Casual;Free to Play;Indie;RPG;Simulation;Sports;Strategy'
 'Casual;Free to Play;Massively Multiplayer;RPG;Early Access'] 1552


In [25]:
print(df_steam["Steamspy Categories"].unique(), len(df_steam["Steamspy Categories"].unique()))

['Action;FPS;Multiplayer' 'FPS;World War II;Multiplayer'
 'FPS;Action;Sci-fi' ... 'Casual;Adventure;Arcade'
 'Free to Play;Visual Novel' 'Early Access;Adventure;Sexual Content'] 6423


In [21]:
# Convert Google Play Categories into same style as Steam
df_googleplay["Category"].unique()

array(['ACTION', 'CASUAL', 'ROLE_PLAYING', 'PUZZLE', 'RACING',
       'ADVENTURE', 'ARCADE', 'STRATEGY', 'SPORTS', 'SIMULATION', 'MUSIC',
       'EDUCATIONAL', 'WORD', 'TRIVIA', 'BOARD', 'CASINO', 'CARD'],
      dtype=object)

In [30]:
# A function that saves arrays into text files
def array2txt(array, file_name = 'output.txt'):
    with open(file_name, "w") as txt_file:
        for i in array:
            txt_file.write(i + "\n")

In [31]:
# Save the uncleaned categories into text files
array2txt(df_steam["Steam Categories"].unique(), 'steam_categories_uncleaned.txt')


In [38]:
# Split the categories connected by semicolons
def split_semi(array):
    output = []
    for i in array:
        output.extend(i.split(';'))
    return output

In [40]:
split_semi(df_steam["Steam Categories"].unique())
len(split_semi(df_steam["Steam Categories"].unique()))

21447

In [36]:
# Filter identical items in a list
def unique(list1): 
    x = np.array(list1) 
    #print(np.unique(x))
    return np.unique(x)

In [37]:
unique(["1","1",'2','3'])

array(['1', '2', '3'], dtype='<U1')

In [42]:
# Cleaned steam categories
array_steam = unique(split_semi(df_steam["Steam Categories"].unique()))
array_steam

array(['Captions available', 'Co-op', 'Commentary available',
       'Cross-Platform Multiplayer', 'Full controller support',
       'In-App Purchases', 'Includes Source SDK', 'Includes level editor',
       'Local Co-op', 'Local Multi-Player', 'MMO', 'Mods',
       'Mods (require HL2)', 'Multi-player', 'Online Co-op',
       'Online Multi-Player', 'Partial Controller Support',
       'Shared/Split Screen', 'Single-player', 'Stats',
       'Steam Achievements', 'Steam Cloud', 'Steam Leaderboards',
       'Steam Trading Cards', 'Steam Turn Notifications',
       'Steam Workshop', 'SteamVR Collectibles', 'VR Support',
       'Valve Anti-Cheat enabled'], dtype='<U26')

In [44]:
# Cleaned steam genres
array_genre = unique(split_semi(df_steam["Genre"].unique()))
array_genre

array(['Accounting', 'Action', 'Adventure', 'Animation & Modeling',
       'Audio Production', 'Casual', 'Design & Illustration',
       'Documentary', 'Early Access', 'Education', 'Free to Play',
       'Game Development', 'Gore', 'Indie', 'Massively Multiplayer',
       'Nudity', 'Photo Editing', 'RPG', 'Racing', 'Sexual Content',
       'Simulation', 'Software Training', 'Sports', 'Strategy',
       'Tutorial', 'Utilities', 'Video Production', 'Violent',
       'Web Publishing'], dtype='<U21')

In [46]:
# Cleaned steamspy categories
array_steampy = unique(split_semi(df_steam["Steamspy Categories"].unique()))
array_steampy

array(['1980s', "1990's", '2.5D', '2D', '2D Fighter', '360 Video', '3D',
       '3D Platformer', '3D Vision', '4 Player Local', '4X', '6DOF',
       'Abstract', 'Action', 'Action RPG', 'Action-Adventure',
       'Adventure', 'Agriculture', 'Aliens', 'Alternate History',
       'America', 'Animation & Modeling', 'Anime', 'Arcade',
       'Arena Shooter', 'Assassin', 'Atmospheric', 'Audio Production',
       'BMX', 'Base-Building', 'Baseball', 'Basketball', 'Batman',
       'Battle Royale', "Beat 'em up", 'Beautiful', 'Benchmark', 'Bikes',
       'Blood', 'Board Game', 'Bowling', 'Building', 'Bullet Hell',
       'Bullet Time', 'CRPG', 'Capitalism', 'Card Game', 'Cartoon',
       'Cartoony', 'Casual', 'Cats', 'Character Action Game',
       'Character Customization', 'Chess', 'Choices Matter',
       'Choose Your Own Adventure', 'Cinematic', 'City Builder',
       'Class-Based', 'Classic', 'Clicker', 'Co-op', 'Cold War',
       'Colorful', 'Comedy', 'Comic Book', 'Competitive', 'Controll

In [47]:
# Save cleaned categories into text files
array2txt(array_steam, 'steam_categories_cleaned.txt')
array2txt(array_genre, 'steam_genres_cleaned.txt')
array2txt(array_steampy, 'steamspy_categories_cleaned.txt')

In [48]:
# Combine all the categories into one single array
array_googleplay = df_googleplay["Category"].unique()
all_categories = np.concatenate((array_steam, array_genre, array_steampy, array_googleplay), axis=0)
print(all_categories, len(all_categories))

['Captions available' 'Co-op' 'Commentary available'
 'Cross-Platform Multiplayer' 'Full controller support' 'In-App Purchases'
 'Includes Source SDK' 'Includes level editor' 'Local Co-op'
 'Local Multi-Player' 'MMO' 'Mods' 'Mods (require HL2)' 'Multi-player'
 'Online Co-op' 'Online Multi-Player' 'Partial Controller Support'
 'Shared/Split Screen' 'Single-player' 'Stats' 'Steam Achievements'
 'Steam Cloud' 'Steam Leaderboards' 'Steam Trading Cards'
 'Steam Turn Notifications' 'Steam Workshop' 'SteamVR Collectibles'
 'VR Support' 'Valve Anti-Cheat enabled' 'Accounting' 'Action' 'Adventure'
 'Animation & Modeling' 'Audio Production' 'Casual'
 'Design & Illustration' 'Documentary' 'Early Access' 'Education'
 'Free to Play' 'Game Development' 'Gore' 'Indie' 'Massively Multiplayer'
 'Nudity' 'Photo Editing' 'RPG' 'Racing' 'Sexual Content' 'Simulation'
 'Software Training' 'Sports' 'Strategy' 'Tutorial' 'Utilities'
 'Video Production' 'Violent' 'Web Publishing' '1980s' "1990's" '2.5D'
 '2D' 

In [52]:
# Also prepare steam and google play as 2 separate groups of categories
all_steam = np.concatenate((array_steam, array_genre, array_steampy), axis=0)
print(all_steam, len(all_steam))
array2txt(all_steam, 'all_steam_categories_cleaned.txt')
array2txt(array_googleplay, 'googleplay_categories.txt')

['Captions available' 'Co-op' 'Commentary available'
 'Cross-Platform Multiplayer' 'Full controller support' 'In-App Purchases'
 'Includes Source SDK' 'Includes level editor' 'Local Co-op'
 'Local Multi-Player' 'MMO' 'Mods' 'Mods (require HL2)' 'Multi-player'
 'Online Co-op' 'Online Multi-Player' 'Partial Controller Support'
 'Shared/Split Screen' 'Single-player' 'Stats' 'Steam Achievements'
 'Steam Cloud' 'Steam Leaderboards' 'Steam Trading Cards'
 'Steam Turn Notifications' 'Steam Workshop' 'SteamVR Collectibles'
 'VR Support' 'Valve Anti-Cheat enabled' 'Accounting' 'Action' 'Adventure'
 'Animation & Modeling' 'Audio Production' 'Casual'
 'Design & Illustration' 'Documentary' 'Early Access' 'Education'
 'Free to Play' 'Game Development' 'Gore' 'Indie' 'Massively Multiplayer'
 'Nudity' 'Photo Editing' 'RPG' 'Racing' 'Sexual Content' 'Simulation'
 'Software Training' 'Sports' 'Strategy' 'Tutorial' 'Utilities'
 'Video Production' 'Violent' 'Web Publishing' '1980s' "1990's" '2.5D'
 '2D' 

# Data Analysis & Results

Include cells that describe the steps in your data analysis.

In [23]:
## YOUR CODE HERE
## FEEL FREE TO ADD MULTIPLE CELLS PER SECTION

# Ethics & Privacy

*Fill in your ethics & privacy discussion here*

# Conclusion & Discussion

*Fill in your discussion information here*